In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
train_df = pd.read_csv('C:\\Users\\SaiMahendraReddyAtlu\\Documents\\Coding Challenge\\DS coding challenge\\train.csv')

In [3]:
test_df = pd.read_csv('C:\\Users\\SaiMahendraReddyAtlu\\Documents\\Coding Challenge\\DS coding challenge\\test.csv')

In [4]:
stores=pd.read_csv("C:\\Users\\SaiMahendraReddyAtlu\\Documents\\Coding Challenge\\DS coding challenge\\stores.csv")

In [5]:
train_df.rename(columns={'Store_no.': 'Store'}, inplace=True)
test_df.rename(columns={'Store_no.': 'Store'}, inplace=True)

In [6]:
train_df = pd.merge(train_df, stores, on='Store', how='inner')
test_df = pd.merge(test_df, stores, on='Store', how='inner')

In [7]:
train_df['Date'] = pd.to_datetime(train_df['Date'], format='%d-%m-%Y')
train_df['year'], train_df['week'], _ = zip(*train_df['Date'].apply(lambda x: x.isocalendar()))
train_df['month'] = train_df['Date'].dt.month

In [8]:
test_df['Date'] = pd.to_datetime(test_df['Date'], format='%d-%m-%Y')
test_df['year'], test_df['week'], _ = zip(*test_df['Date'].apply(lambda x: x.isocalendar()))
test_df['month'] = test_df['Date'].dt.month

In [9]:
train_df.drop('Date',axis=1,inplace=True)

In [10]:
train_df = pd.get_dummies(train_df, columns=['Type'],drop_first=True)
test_df = pd.get_dummies(test_df, columns=['Type'], drop_first=True)

In [11]:
train_df.loc[train_df['Weekly_sales'] < 0, 'Weekly_sales'].value_counts()

-15.00     25
-5.00      14
-2.00      13
-4.00       9
-12.00      9
           ..
-5.12       1
-42.78      1
-465.08     1
-90.00      1
-0.24       1
Name: Weekly_sales, Length: 408, dtype: int64

In [12]:
train_df=train_df[train_df['Weekly_sales']>0]

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_df.drop(columns=['Weekly_sales']), 
                                                  train_df['Weekly_sales'], 
                                                  test_size=0.2, 
                                                  random_state=42)


In [14]:

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred)
print(f"MAPE for Random Forest Regression: {mape}")

MAPE for Random Forest Regression: 0.654504155143873


In [15]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [16]:
from xgboost import XGBRegressor as xgb
x = xgb(random_state=42)
x.fit(X_train, y_train)

y_pred = x.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred)
print(f"MAPE for XGBoosting Regression: {mape}")

MAPE for XGBoosting Regression: 26.326513378238793


In [17]:
x=test_df.drop(['Date'],axis=1)
test_pred=rf_model.predict(x)
test_df['Weekly_sales']=test_pred

In [18]:
test_df

,Store,Department,Date,Holiday_ind,Size,year,week,month,Type_B,Type_C,Weekly_sales
0,1,1,2018-08-03,0,151315,2018,31,8,0,0,16383.8111
1,1,1,2018-08-10,0,151315,2018,32,8,0,0,15882.6427
2,1,1,2018-08-17,0,151315,2018,33,8,0,0,15888.7380
3,1,1,2018-08-24,0,151315,2018,34,8,0,0,15852.0714
4,1,1,2018-08-31,0,151315,2018,35,8,0,0,15931.9417
...,...,...,...,...,...,...,...,...,...,...,...
22100,30,83,2018-09-28,0,118221,2018,39,9,1,0,472.3450
22101,30,83,2018-10-05,0,118221,2018,40,10,1,0,524.2162
22102,30,83,2018-10-12,0,118221,2018,41,10,1,0,533.6894
22103,30,83,2018-10-19,0,118221,2018,42,10,1,0,581.9308


In [19]:
submission=test_df.drop(['Holiday_ind','Size','year','week','month','Type_B','Type_C'],axis=1)

In [20]:
submission

,Store,Department,Date,Weekly_sales
0,1,1,2018-08-03,16383.8111
1,1,1,2018-08-10,15882.6427
2,1,1,2018-08-17,15888.7380
3,1,1,2018-08-24,15852.0714
4,1,1,2018-08-31,15931.9417
...,...,...,...,...
22100,30,83,2018-09-28,472.3450
22101,30,83,2018-10-05,524.2162
22102,30,83,2018-10-12,533.6894
22103,30,83,2018-10-19,581.9308


In [21]:
submission.to_csv('csv_submission.csv', index=False)